# Model Training: Global SARIMA

Ce notebook entraîne un modèle SARIMA global sur les données agrégées (tous produits/magasins confondus).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import os
import warnings

warnings.filterwarnings('ignore')

os.makedirs('trained_models', exist_ok=True)
print("✅ Bibliothèques importées.")

## Chargement des Données
Nous chargeons les données déjà agrégées et divisées en Train/Test.

In [ ]:
train_path = 'train_sarimax.csv'
test_path = 'test_sarimax.csv'

if not os.path.exists(train_path) or not os.path.exists(test_path):
    raise FileNotFoundError("Fichiers train_sarimax.csv ou test_sarimax.csv manquants.")

# Chargement avec Date en index
train_df = pd.read_csv(train_path, parse_dates=['Date'], index_col='Date')
test_df = pd.read_csv(test_path, parse_dates=['Date'], index_col='Date')

# S'assurer que la fréquence est définie (Hebdomadaire 'W-SUN' par défaut souvent)
train_df.index.freq = 'W'
test_df.index.freq = 'W'

print(f"📦 Train shape: {train_df.shape}")
print(f"📦 Test shape: {test_df.shape}")

In [ ]:
# Visualisation de la série
plt.figure(figsize=(12, 5))
plt.plot(train_df['Units Sold'], label='Train')
plt.plot(test_df['Units Sold'], label='Test')
plt.title("Ventes Globales (Units Sold)")
plt.legend()
plt.show()

## Entraînement SARIMA Global
Nous utilisons `Units Sold` comme cible et les autres colonnes comme variables exogènes (`Price`, `Discount`, `Holiday/Promotion`, `Competitor Pricing`).

In [ ]:
target_col = 'Units Sold'
exog_cols = ['Price', 'Discount', 'Holiday/Promotion', 'Competitor Pricing']

# Préparation y et X (exog)
y_train = train_df[target_col]
X_train = train_df[exog_cols]

y_test = test_df[target_col]
X_test = test_df[exog_cols]

print("⏳ Entraînement SARIMAX...")

# Configuration du modèle
# Order (p,d,q) = (1,1,1) standard pour commencer
# Seasonal Order (P,D,Q,s) = (1,1,1,52) pour une saisonnalité annuelle sur données hebdos
# Attention: s=52 est lourd. Essayons s=4 (mensuel approx) ou s=12 (trimestriel) pour la démo si s=52 trop lent.
# Pour l'exercice, on va utiliser s=12 pour représenter une 'saisonnalité' trimestrielle/saisonnière si les données ne couvrent pas assez d'années.
# Si on a > 2 ans de données, s=52 est mieux.

model = SARIMAX(
    y_train,
    exog=X_train,
    order=(1, 1, 1),
    seasonal_order=(0, 1, 1, 52), # Tentative s=52
    enforce_stationarity=False,
    enforce_invertibility=False
)

model_fit = model.fit(disp=False)
print("✅ Modèle entraîné.")
print(model_fit.summary())

## Évaluation et Prévisions

In [ ]:
# Prévision
preds = model_fit.forecast(steps=len(test_df), exog=X_test)
preds = pd.Series(preds, index=test_df.index)

# Métriques
rmse = np.sqrt(mean_squared_error(y_test, preds))
mae = mean_absolute_error(y_test, preds)

print(f"RMSE: {rmse:.2f}")
print(f"MAE:  {mae:.2f}")

# Plot
plt.figure(figsize=(12, 6))
plt.plot(train_df['Units Sold'], label='Train')
plt.plot(y_test, label='Test (Réel)')
plt.plot(preds, label='Prévision', linestyle='--')
plt.title("Prévision SARIMAX Global")
plt.legend()
plt.show()

In [ ]:
# Sauvegarde
model_path = 'trained_models/global_sarima.pkl'
joblib.dump(model_fit, model_path)
print(f"💾 Modèle sauvegardé : {model_path}")